# Calorimetry likelihood stopping muons with arrays

In [1]:
%load_ext autoreload
%autoreload 2

import math
from itertools import product
import uproot
import awkward
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.stats import binned_statistic

import matplotlib
matplotlib.rcParams.update({'font.size': 12})
from calo_likelihood import caloLikelihood
from general_functions import *
from loaders import *
from plotter import *
from categorisation import *
from initializers import *

In [2]:
# today = 'calorimetry'
today = 'calorimetry_trial2'

!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/muons/low_rr_before_after/
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/muons/low_rr_before_after/plane0/
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/muons/low_rr_before_after/plane1/
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/muons/low_rr_before_after/plane2/
save_folder_low_rr = '/home/nic/Dropbox/MicroBooNE/_fig/' + today + '/muons/low_rr_before_after/'

In [3]:
branches = [
    'run',
    'sub',
    'evt',
    
    'backtracked_pdg',
    'backtracked_e',
    'backtracked_purity',
    'backtracked_completeness',
    'backtracked_overlay_purity',
    'backtracked_end_process',
    
    'backtracked_start_x',
    'backtracked_start_y',
    'backtracked_start_z',
    
    'backtracked_sce_start_x',
    'backtracked_sce_start_y',
    'backtracked_sce_start_z',
    
    'nplanehits_U',
    'nplanehits_V',
    'nplanehits_Y',
    'trk_score',

    'generation',
    'trk_daughters',
    'shr_daughters',

    'trk_sce_start_x',
    'trk_sce_start_y',
    'trk_sce_start_z',

    'trk_sce_end_x',
    'trk_sce_end_y',
    'trk_sce_end_z',
    
    'trk_theta',
    'trk_phi',

    'trk_dir_x',
    'trk_dir_y',
    'trk_dir_z',


    'trk_len',
    'trk_distance',
    'longest',
    
    'trk_mcs_muon_mom',
    'trk_range_muon_mom',
    
    'is_hit_montecarlo_u',
    'is_hit_montecarlo_v',
    'is_hit_montecarlo_y',
    
    'dqdx_u',
    'dqdx_v',
    'dqdx_y',

#     'dedx_u',
#     'dedx_v',
#     'dedx_y',

    'rr_u',
    'rr_v',
    'rr_y',

    'pitch_u',
    'pitch_v',
    'pitch_y',
    
    'dir_x_u',
    'dir_x_v',
    'dir_x_y',
    
    'dir_y_u',
    'dir_y_v',
    'dir_y_y',
    
    'dir_z_u',
    'dir_z_v',
    'dir_z_y',
]

In [4]:
directory = '/home/nic/Desktop/MicroBooNE/calorimetry_likelihood/'
pot_beam_on = 4.054e+19
n_triggers_on = 8977290.0
n_triggers_off = 33240727.0

fraction = 1.
pot_beam_on *= fraction

file_lists = {}

file_lists['beam_on'] = directory+'v07/beam_on/out_muon_skimmed.root'
file_lists['beam_off'] = directory+'v07/beam_off/out_muon_skimmed.root'
file_lists['bnb_nu'] = directory+'v08/bnb_nu/out_muon_skimmed.root'
file_lists['bnb_dirt'] = directory+'v07/bnb_dirt/out_muon_skimmed.root'

folder = "nuselection"
events_tree_name = "CalorimetryAnalyzer"
pot_tree_name = "SubRun"

In [5]:
arrays, pot = load_data_calo(file_lists, 
                        branches, 
                        folder, 
                        events_tree_name, 
                        pot_tree_name, 
                        labels=['beam_on', 'beam_off', 'bnb_nu', 'bnb_dirt'],
                        lazy=False,
                        fraction=fraction)

start loading beam_on
start loading beam_off
start loading bnb_nu
start loading bnb_dirt
Done!


In [6]:
scale_factors = compute_scale_factors(pot, pot_beam_on, n_triggers_on, n_triggers_off)

In [7]:
for name, array in arrays.items():
    print(name)
    initialisation_for_caloriemtry_data_mc(array)

beam_on
beam_off
bnb_nu
bnb_dirt


In [8]:
upr = uproot.open(file_lists['bnb_nu'])[folder]['NeutrinoSelectionFilter']

In [9]:
aux = upr.arrays(branches='weight*', namedecode="utf-8")

In [10]:
aux.keys()

dict_keys(['weightSpline', 'weightTune', 'weightSplineTimesTune'])

In [11]:
mask = (aux['weightSpline'] * aux['weightTune'] - aux['weightSplineTimesTune'])!= 0

/home/nic/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in subtract
  """Entry point for launching an IPython kernel.


In [12]:
aux['weightSpline'][mask]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)

In [13]:
aux['weightTune'][mask]

array([inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf], dtype=float32)

In [14]:
aux['weightSplineTimesTune'][mask]

array([inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf], dtype=float32)

# Base plots data/MC comparison 

In [15]:
my_plotter = plotter(arrays, scale_factors, pot_beam_on, branch_weights_name='weightSplineTimesTune')
title_base = 'Longest track, trk-score > 0.5\ntrack length > 30 cm, start-end fiducial\ntrk-vtx distance < 5 cm, muon-momentum-consistency < 0.5'

In [16]:
variables = [
    'trk_len',
    'trk_score',
    'trk_muon_mom_consistency',
    'trk_theta',
    'trk_phi'
]

binnings = {
    'trk_len': [20, 0, 600],
    'trk_score': [50, 0, 1],
    'trk_muon_mom_consistency': [50, -0.5, 0.5],
    'trk_theta': [50, 0, np.pi],
    'trk_phi': [50, -np.pi, np.pi]
}

xlabels = {
    'trk_len': 'Track length [cm]',
    'trk_score': r'Track score',
    'trk_muon_mom_consistency': r'Muon Momentum Consistency',
    'trk_theta': r'Track $\theta$',
    'trk_phi': r'Track $\phi$'
}

In [17]:
for name, array in arrays.items():
    print(name, array['weight'].sum())

beam_on 2876.0
beam_off 560.6632502351708
bnb_nu 2064.2682306708143
bnb_dirt 86.50941924219083


In [18]:
for name, array in arrays.items():
    print(name, array['weight'].sum())

beam_on 2876.0
beam_off 560.6632502351708
bnb_nu 2064.2682306708143
bnb_dirt 86.50941924219083


In [19]:
num = arrays['bnb_nu']['weight'][(np.abs(arrays['bnb_nu']['backtracked_pdg'])==13)].sum() + arrays['bnb_dirt']['weight'][(np.abs(arrays['bnb_dirt']['backtracked_pdg'])==13)].sum()
den = arrays['bnb_nu']['weight'].sum() + arrays['bnb_dirt']['weight'].sum() + arrays['beam_off']['weight'].sum()

purity = num/den
print(num, den)
print('Purity with beam off: {:.2f}'.format(purity))

1588.0814619713096 2711.440900148176
Purity with beam off: 0.59


In [20]:
num = arrays['bnb_nu']['weight'][(np.abs(arrays['bnb_nu']['backtracked_pdg'])==13)].sum() + arrays['bnb_dirt']['weight'][(np.abs(arrays['bnb_dirt']['backtracked_pdg'])==13)].sum()
den = arrays['bnb_nu']['weight'].sum() + arrays['bnb_dirt']['weight'].sum()

purity = num/den
print(num, den)
print('Purity without beam off: {:.2f}'.format(purity))

1588.0814619713096 2150.777649913005
Purity without beam off: 0.74


In [21]:
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/muons/base_plots/
save_folder_base = '/home/nic/Dropbox/MicroBooNE/_fig/' + today + '/muons/base_plots/'

for var in variables:
    out = my_plotter.plot_pot_normalised_var(variable=var,
                                   binning=binnings[var], 
                                   categories=pdg_categories, 
                                   additional_selection=None, 
                                   prediction_datasets=['bnb_dirt', 'bnb_nu'],
                                   xlabel=xlabels[var], 
                                   title=title_base,
                                   log=False, 
                                   subtracted=False, 
                                   onebin=False)
    plt.savefig(save_folder_base + var + '_pot.png', dpi=300)
    plt.savefig(save_folder_base + var + '_pot.pdf')
    plt.close()
    
    
    out = my_plotter.plot_pot_normalised_var(variable=var,
                                   binning=binnings[var], 
                                   categories=pdg_categories, 
                                   additional_selection=None, 
                                   prediction_datasets=['bnb_dirt', 'bnb_nu'],
                                   xlabel=xlabels[var], 
                                   title=title_base,
                                   log=False, 
                                   subtracted=True, 
                                   onebin=False,
                                   area_norm=True)
    plt.savefig(save_folder_base + var + '_area_norm.png', dpi=300)
    plt.savefig(save_folder_base + var + '_area_norm.pdf')
    plt.close()

/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/plotter.py:182: RuntimeWarning: invalid value encountered in true_divide
  bin_contents_ratio = beam_on_values / bin_contents_prediction[-1]
/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/plotter.py:183: RuntimeWarning: invalid value encountered in true_divide
  bin_relative_errors_prediction = bin_errors_prediction / bin_contents_prediction[-1]
/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/plotter.py:184: RuntimeWarning: invalid value encountered in true_divide
  bin_relative_errors_data = beam_on_y_err / beam_on_values
/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/plotter.py:208: RuntimeWarning: invalid value encountered in true_divide
  chi2_bins = (beam_on_values - bin_contents_prediction[-1])**2 / chi2_errors
/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/plotter.py:182: RuntimeWarning: invalid value encountered in true_divide
  bin_contents_ratio = beam_on_values / bin_contents_prediction[-1]
/home/nic/Dropbox/MicroB

# Plot dqdx all hits

In [15]:
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/muons/base_plots/
save_folder_base = '/home/nic/Dropbox/MicroBooNE/_fig/' + today + '/muons/base_plots/'

dedx_vars = 'dqdx'
binning = [60, 0, 600]

for plane_num, plane in zip([2, 1, 0], ['_y', '_v', '_u']):
    def sel_function(array):
                return (array['first_last_hit_mask'+plane])
    title = ', all muon candidates\n' + title_base + '\nplane {}'.format(plane_num)

    save_label = '_integrated_dqdx_aea_norm_'

    out = my_plotter.plot_pot_normalised_var(variable='dqdx'+plane,
                               binning=binning, 
                               categories=pdg_categories, 
                               additional_selection=sel_function, 
                               prediction_datasets=['bnb_dirt', 'bnb_nu'],
                               xlabel=r'Hit dQ/dx [ADC/cm]', 
                               title="Before re-calibration" + title,
                               log=False,
                               subtracted=True,
                               area_norm=True,
                               onebin=False)
    plt.savefig(save_folder_base + 'plane_{}'.format(plane_num) + save_label + 'avant.png', dpi=300)
    plt.savefig(save_folder_base + 'plane_{}'.format(plane_num) + save_label + 'avant.pdf')
    plt.close()

    out2 = my_plotter.plot_pot_normalised_var(variable='dqdx'+plane+"_cali",
                               binning=binning, 
                               categories=pdg_categories, 
                               additional_selection=sel_function, 
                               prediction_datasets=['bnb_dirt', 'bnb_nu'],
                               xlabel=r'Calibrated Hit dQ/dx [ADC/cm]', 
                               title="After re-calibration" + title,
                               log=False,
                               subtracted=True,
                               area_norm=True,
                               onebin=False)
    plt.savefig(save_folder_base + 'plane_{}'.format(plane_num) + save_label + 'depois.png', dpi=300)
    plt.savefig(save_folder_base + 'plane_{}'.format(plane_num) + save_label + 'depois.pdf')
    plt.close()
    
    save_label = '_integrated_dqdx_pot_norm'
    
    out3 = my_plotter.plot_pot_normalised_var(variable='dqdx'+plane,
                               binning=binning, 
                               categories=pdg_categories, 
                               additional_selection=sel_function, 
                               prediction_datasets=['bnb_dirt', 'bnb_nu'],
                               xlabel=r'Hit dQ/dx [ADC/cm]', 
                               title="Before re-calibration" + title,
                               log=False,
                               subtracted=False,
                               area_norm=False,
                               onebin=False)
    plt.savefig(save_folder_base + 'plane_{}'.format(plane_num) + save_label + 'avant.png', dpi=300)
    plt.savefig(save_folder_base + 'plane_{}'.format(plane_num) + save_label + 'avant.pdf')
    plt.close()

    out4 = my_plotter.plot_pot_normalised_var(variable='dqdx'+plane+"_cali",
                               binning=binning, 
                               categories=pdg_categories, 
                               additional_selection=sel_function, 
                               prediction_datasets=['bnb_dirt', 'bnb_nu'],
                               xlabel=r'Calibrated Hit dQ/dx [ADC/cm]', 
                               title="After re-calibration" + title,
                               log=False,
                               subtracted=False,
                               area_norm=False,
                               onebin=False)
    plt.savefig(save_folder_base + 'plane_{}'.format(plane_num) + save_label + 'depois.png', dpi=300)
    plt.savefig(save_folder_base + 'plane_{}'.format(plane_num) + save_label + 'depois.pdf')
    plt.close()

# Plots in bins of rr only

In [16]:
this_folder = 'rr_only'
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/muons/$this_folder/
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/muons/$this_folder/plane0/
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/muons/$this_folder/plane1/
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/muons/$this_folder/plane2/
this_save_folder = '/home/nic/Dropbox/MicroBooNE/_fig/' + today + '/muons/{}/'.format(this_folder)

parameter = 'rr'
parameter_legend_names = 'residual range'
parameter_bin_edges = [0., 2, 5, 10, 20, 30, 300]
parameter_bin_centers = (np.array(parameter_bin_edges[:-1]) + np.array(parameter_bin_edges[1:]))/2
dedx_vars = 'dqdx'
binning = [60, 0, 1000]

chi2_avant = {}
chi2_depois = {}

# for plane_num, plane in zip([2, 1, 0], ['_y', '_v', '_u']):
for plane_num, plane in zip([0, 1, 2], ['_u', '_v', '_y']):
    chi2_avant[plane_num] = []
    chi2_depois[plane_num] = []
    for par0_edges in zip(parameter_bin_edges[:-1], parameter_bin_edges[1:]):
        this_parameter = parameter+plane
        def sel_function(array):
            return (array['first_last_hit_mask'+plane]) &\
                    (array[this_parameter] > par0_edges[0]) &\
                    (array[this_parameter] <= par0_edges[1])
        title = title_base + '\nplane {}, {:.2f} < {} < {:.2f}'.format(plane_num, 
                                                                       par0_edges[0], 
                                                                       parameter_legend_names,
                                                                       par0_edges[1])

        save_label = '{:04.0f}_{}_{:04.0f}'.format(par0_edges[0], 
                                                 parameter_legend_names,
                                                 par0_edges[1]).replace('.', '').replace('$', '').replace('\\', '')

        out = my_plotter.plot_pot_normalised_var(variable='dqdx'+plane,
                                   binning=binning, 
                                   categories=pdg_categories, 
                                   additional_selection=sel_function, 
                                   prediction_datasets=['bnb_dirt', 'bnb_nu'],
                                   xlabel=r'Hit dQ/dx [ADC/cm]', 
                                   title="Before re-calibration\n" + title,
                                   log=False,
                                   subtracted=True,
                                   area_norm=True,
                                   onebin=False)
        plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_avant.png', dpi=300)
        plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_avant.pdf')
        plt.close()
        
        chi2_avant[plane_num].append(out[1]/out[2])
        
        out = my_plotter.plot_pot_normalised_var(variable='dqdx'+plane,
                                   binning=binning, 
                                   categories=pdg_categories, 
                                   additional_selection=sel_function, 
                                   prediction_datasets=['bnb_dirt', 'bnb_nu'],
                                   xlabel=r'Hit dQ/dx [ADC/cm]', 
                                   title="Before re-calibration\n" + title,
                                   log=False,
                                   subtracted=True,
                                   area_norm=True,
                                   onebin=False,
                                   legend=False)
        plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_noleg_avant.png', dpi=300)
        plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_noleg_avant.pdf')
        plt.close()

        out2 = my_plotter.plot_pot_normalised_var(variable='dqdx'+plane+"_cali",
                                   binning=binning, 
                                   categories=pdg_categories, 
                                   additional_selection=sel_function, 
                                   prediction_datasets=['bnb_dirt', 'bnb_nu'],
                                   xlabel=r'Calibrated Hit dQ/dx [ADC/cm]', 
                                   title="After re-calibration\n" + title,
                                   log=False,
                                   subtracted=True,
                                   area_norm=True,
                                   onebin=False)
        plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_depois.png', dpi=300)
        plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_depois.pdf')
        plt.close()
        
        chi2_depois[plane_num].append(out2[1]/out2[2])
        
        out2 = my_plotter.plot_pot_normalised_var(variable='dqdx'+plane+"_cali",
                                   binning=binning, 
                                   categories=pdg_categories, 
                                   additional_selection=sel_function, 
                                   prediction_datasets=['bnb_dirt', 'bnb_nu'],
                                   xlabel=r'Calibrated Hit dQ/dx [ADC/cm]', 
                                   title="After re-calibration\n" + title,
                                   log=False,
                                   subtracted=True,
                                   area_norm=True,
                                   onebin=False,
                                   legend=False)
        plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_noleg_depois.png', dpi=300)
        plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_noleg_depois.pdf')
        plt.close()

In [17]:
# summary plots of the chi2
plt.rcParams.update({'font.size': 11})
title = 'Effect of the recalibration\n' + title_base
colours = ['orange', 'blue', 'green']

def set_ticks():
    plt.xticks(range(len(parameter_bin_centers)), parameter_bin_centers)
    plt.xlabel("residual range [cm]")
    plt.legend()
    plt.title(title, loc='left')
    plt.tight_layout()
    
    
for plane in range(3):
    plt.plot(chi2_avant[plane], '--', color=colours[plane], label='plane {}, before recalibration'.format(plane))
    plt.plot(chi2_depois[plane], '-', color=colours[plane], label='plane {}, after recalibration'.format(plane))

plt.ylabel(r"$X^2$/ndof")
set_ticks()
plt.savefig(this_save_folder+'chi2.png', dpi=250)
plt.savefig(this_save_folder+'chi2.pdf', dpi=250)
plt.close()


for plane in range(3):
    plt.plot(np.array(chi2_depois[plane])/np.array(chi2_avant[plane]), '-', color=colours[plane], label='plane {}'.format(plane))
    
plt.ylabel(r"$X^2_{after}/X^2_{before}$")
set_ticks()
plt.savefig(this_save_folder+'chi2_ratio.png', dpi=250)
plt.savefig(this_save_folder+'chi2_ratio.pdf', dpi=250)
plt.close()

for plane in range(3):
    plt.plot(np.array(chi2_depois[plane]) - np.array(chi2_avant[plane]), '-', color=colours[plane], label='plane {}'.format(plane))
    
plt.ylabel(r"$\Delta X^2$/ndof")
set_ticks()
plt.savefig(this_save_folder+'chi2_diff.png', dpi=250)
plt.savefig(this_save_folder+'chi2_diff.pdf', dpi=250)
plt.close()

# Plots at high rr, in bins of pitch and theta_yx

In [24]:
this_folder = 'high_rr_before_after'
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/muons/$this_folder/
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/muons/$this_folder/plane0/
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/muons/$this_folder/plane1/
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/muons/$this_folder/plane2/
this_save_folder = '/home/nic/Dropbox/MicroBooNE/_fig/' + today + '/muons/{}/'.format(this_folder)

parameters = ['pitch', 'abs_theta_yx']
parameters_legend_names = ['pitch', r'$\phi$']
parameters_bin_edges = [np.array([0.3, 0.4, 0.7, 1., 1.5, 30]), np.linspace(0, np.pi/2, 7)]

dedx_vars = 'dqdx'
binning = [60, 0, 600]

chi2_high_rr_avant = {}
chi2_high_rr_depois = {}

# for plane_num, plane in zip([2, 1, 0], ['_y', '_v', '_u']):
for plane_num, plane in zip([0, 1, 2], ['_u', '_v', '_y']):
    chi2_high_rr_avant[plane_num] = []
    chi2_high_rr_depois[plane_num] = []
    for par0_edges in zip(parameters_bin_edges[0][:-1], parameters_bin_edges[0][1:]):
        chi2_high_rr_avant[plane_num].append([])
        chi2_high_rr_depois[plane_num].append([])
        for par1_edges in zip(parameters_bin_edges[1][:-1], parameters_bin_edges[1][1:]):
            this_parameters = [par+plane for par in parameters]
            def sel_function(array):
                return (array['first_last_hit_mask'+plane]) &\
                       (array['rr'+plane] > 30) &\
                        ((array[this_parameters[0]]) > par0_edges[0]) &\
                        ((array[this_parameters[0]]) <= par0_edges[1]) &\
                        ((array[this_parameters[1]]) > par1_edges[0]) &\
                        ((array[this_parameters[1]]) <= par1_edges[1])
            title = title_base + '\nplane {}, {:2.2f} < {} < {:2.2f} & {:2.2f} < {} < {:2.2f}'.format(plane_num, par0_edges[0], 
                                                         parameters_legend_names[0],
                                                         par0_edges[1],
                                                         par1_edges[0], 
                                                         parameters_legend_names[1],
                                                         par1_edges[1],)
            
#             save_label = '{:04.1f}_{}_{:04.1f}_{:04.2f}_{}_{:04.2f}'.format( 
#                                                          par0_edges[0], 
#                                                          parameters[0],
#                                                          par0_edges[1],
#                                                          par1_edges[0], 
#                                                          parameters[1],
#                                                          par1_edges[1]).replace('.', '').replace('$', '').replace('\\', '')
            
            out = my_plotter.plot_pot_normalised_var(variable='dqdx'+plane,
                                       binning=binning, 
                                       categories=pdg_categories, 
                                       additional_selection=sel_function, 
                                       prediction_datasets=['bnb_dirt', 'bnb_nu'],
                                       xlabel=r'Hit dQ/dx [ADC/cm]', 
                                       title="Before re-calibration, residual range > 30 cm\n" + title,
                                       log=False,
                                       subtracted=True,
                                       area_norm=True,
                                       onebin=False)
#             plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_avant.png', dpi=300)
#             plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_avant.pdf')
            plt.close()
            
            chi2_high_rr_avant[plane_num][-1].append(out[1]/out[2])
            
#             out = my_plotter.plot_pot_normalised_var(variable='dqdx'+plane,
#                                        binning=binning, 
#                                        categories=pdg_categories, 
#                                        additional_selection=sel_function, 
#                                        prediction_datasets=['bnb_dirt', 'bnb_nu'],
#                                        xlabel=r'Hit dQ/dx [ADC/cm]', 
#                                        title="Before re-calibration, residual range > 30 cm\n" + title,
#                                        log=False,
#                                        subtracted=True,
#                                        area_norm=True,
#                                        onebin=False,
#                                        legend=False)
#             plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_noleg_avant.png', dpi=300)
#             plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_noleg_avant.pdf')
#             plt.close()
            
            out2 = my_plotter.plot_pot_normalised_var(variable='dqdx'+plane+"_cali",
                                       binning=binning, 
                                       categories=pdg_categories, 
                                       additional_selection=sel_function, 
                                       prediction_datasets=['bnb_dirt', 'bnb_nu'],
                                       xlabel=r'Calibrated Hit dQ/dx [ADC/cm]', 
                                       title="After re-calibration, residual range > 30 cm\n" + title,
                                       log=False,
                                       subtracted=True,
                                       area_norm=True,
                                       onebin=False)
#             plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_depois.png', dpi=300)
#             plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_depois.pdf')
            plt.close()
            
            chi2_high_rr_depois[plane_num][-1].append(out2[1]/out2[2])

#             out2 = my_plotter.plot_pot_normalised_var(variable='dqdx'+plane+"_cali",
#                                        binning=binning, 
#                                        categories=pdg_categories, 
#                                        additional_selection=sel_function, 
#                                        prediction_datasets=['bnb_dirt', 'bnb_nu'],
#                                        xlabel=r'Calibrated Hit dQ/dx [ADC/cm]', 
#                                        title="After re-calibration, residual range > 30 cm\n" + title,
#                                        log=False,
#                                        subtracted=True,
#                                        area_norm=True,
#                                        onebin=False,
#                                        legend=False)
#             plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_noleg_depois.png', dpi=300)
#             plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_noleg_depois.pdf')
#             plt.close()

/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/plotter.py:184: RuntimeWarning: invalid value encountered in true_divide
  bin_relative_errors_data = beam_on_y_err / beam_on_values
/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/plotter.py:182: RuntimeWarning: invalid value encountered in true_divide
  bin_contents_ratio = beam_on_values / bin_contents_prediction[-1]
/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/plotter.py:183: RuntimeWarning: invalid value encountered in true_divide
  bin_relative_errors_prediction = bin_errors_prediction / bin_contents_prediction[-1]
/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/plotter.py:208: RuntimeWarning: invalid value encountered in true_divide
  chi2_bins = (beam_on_values - bin_contents_prediction[-1])**2 / chi2_errors
/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/plotter.py:182: RuntimeWarning: divide by zero encountered in true_divide
  bin_contents_ratio = beam_on_values / bin_contents_prediction[-1]
/home/nic/Dropbox/Micro

In [26]:
# summary plots of the chi2
plt.rcParams.update({'font.size': 10})

this_title_base = '\n' + title_base + '\nplane {}\nMicroBooNE Preliminary'

def chi2_plot(table, title, save_name):
    plt.pcolormesh(table)
    plt.title(title, loc='left')
    plt.colorbar()
    plt.xticks(range(len(parameters_bin_edges[1])), ['{:.2g}'.format(p) for p in parameters_bin_edges[1]])
    plt.yticks(range(len(parameters_bin_edges[0])), ['{:.2g}'.format(p) for p in parameters_bin_edges[0]])
    plt.xlabel(r"$\phi$")
    plt.ylabel("pitch [cm]")
    
    for i in range(len(parameters_bin_edges[1])-1):
        for j in range(len(parameters_bin_edges[0])-1):
            text = plt.text(i+0.5, j+0.5, "{:.3g}".format(table[j, i]),
                           ha="center", va="center", color="k")
    plt.tight_layout()
    plt.savefig(save_name + '.png', dpi=300)
    plt.savefig(save_name + '.pdf')
    plt.close()

for plane_num in range(3):
    chi2_av = np.array(chi2_high_rr_avant[plane_num])
    chi2_dep = np.array(chi2_high_rr_depois[plane_num])
    
    chi2_plot(chi2_av, title=r"$X^2$" + " Before re-calibration, residual range > 30 cm" + this_title_base.format(plane_num), 
                       save_name=this_save_folder + 'plane_{}_chi2_avant'.format(plane_num))
    
    chi2_plot(chi2_dep, title=r"$X^2$" + " After re-calibration, residual range > 30 cm" + this_title_base.format(plane_num), 
                       save_name=this_save_folder + 'plane_{}_chi2_depois'.format(plane_num))
    
    chi2_plot(chi2_dep-chi2_av, title=r"$X^2_{after} - X^2_{before}/ndof$"+", residual range > 30 cm" + this_title_base.format(plane_num), 
                       save_name=this_save_folder + 'plane_{}_delta_chi2'.format(plane_num))

    chi2_plot(chi2_dep/chi2_av, r"$X^2_{after}/X^2_{before}$"+", residual range > 30 cm" + this_title_base.format(plane_num), 
                       save_name=this_save_folder + 'plane_{}_chi2_ratio'.format(plane_num))

# Plots at low rr, in bins of rr and pitch

In [21]:
this_folder = 'low_rr_before_after'
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/muons/$this_folder/
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/muons/$this_folder/plane0/
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/muons/$this_folder/plane1/
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/muons/$this_folder/plane2/
this_save_folder = '/home/nic/Dropbox/MicroBooNE/_fig/' + today + '/muons/{}/'.format(this_folder)

parameters = ['rr', 'pitch']
parameters_legend_names = ['residual range', 'pitch']
parameters_bin_edges = [[0., 2, 5, 10, 20, 30, 300], [0.3, 0.4, 1, 30]]
dedx_vars = 'dqdx'
binning = [60, 0, 1000]

chi2_avant = {}
chi2_depois = {}

for plane_num, plane in zip([2, 1, 0], ['_y', '_v', '_u']):
    chi2_avant[plane_num] = []
    chi2_depois[plane_num] = []
    for par0_edges in zip(parameters_bin_edges[0][:-1], parameters_bin_edges[0][1:]):
        chi2_avant[plane_num].append([])
        chi2_depois[plane_num].append([])
        for par1_edges in zip(parameters_bin_edges[1][:-1], parameters_bin_edges[1][1:]):
            this_parameters = [par+plane for par in parameters]
            def sel_function(array):
                return (array['first_last_hit_mask'+plane]) &\
                        ((array[this_parameters[0]]) > par0_edges[0]) &\
                        ((array[this_parameters[0]]) <= par0_edges[1]) &\
                        ((array[this_parameters[1]]) > par1_edges[0]) &\
                        ((array[this_parameters[1]]) <= par1_edges[1])
            title = title_base + '\nplane {}, {:.2f} < {} < {:.2f} & {:.2f} < {} < {:.2f}'.format(plane_num, par0_edges[0], 
                                                         parameters_legend_names[0],
                                                         par0_edges[1],
                                                         par1_edges[0], 
                                                         parameters_legend_names[1],
                                                         par1_edges[1],)
            
            save_label = '{:04.0f}_{}_{:04.0f}_{:04.1f}_{}_{:04.1f}'.format( 
                                                         par0_edges[0], 
                                                         parameters[0],
                                                         par0_edges[1],
                                                         par1_edges[0], 
                                                         parameters[1],
                                                         par1_edges[1]).replace('.', '').replace('$', '').replace('\\', '')
            
            out = my_plotter.plot_pot_normalised_var(variable='dqdx'+plane,
                                       binning=binning, 
                                       categories=pdg_categories, 
                                       additional_selection=sel_function, 
                                       prediction_datasets=['bnb_dirt', 'bnb_nu'],
                                       xlabel=r'Hit dQ/dx [ADC/cm]', 
                                       title="Before re-calibration\n" + title,
                                       log=False,
                                       subtracted=True,
                                       area_norm=True,
                                       onebin=False)
            plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_avant.png', dpi=300)
            plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_avant.pdf')
            plt.close()
            
            chi2_avant[plane_num][-1].append(out[1]/out[2])
            
            out = my_plotter.plot_pot_normalised_var(variable='dqdx'+plane,
                                       binning=binning, 
                                       categories=pdg_categories, 
                                       additional_selection=sel_function, 
                                       prediction_datasets=['bnb_dirt', 'bnb_nu'],
                                       xlabel=r'Hit dQ/dx [ADC/cm]', 
                                       title="Before re-calibration\n" + title,
                                       log=False,
                                       subtracted=True,
                                       area_norm=True,
                                       onebin=False,
                                       legend=False)
            plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_noleg_avant.png', dpi=300)
            plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_noleg_avant.pdf')
            plt.close()
            
            
            
            out2 = my_plotter.plot_pot_normalised_var(variable='dqdx'+plane+"_cali",
                                       binning=binning, 
                                       categories=pdg_categories, 
                                       additional_selection=sel_function, 
                                       prediction_datasets=['bnb_dirt', 'bnb_nu'],
                                       xlabel=r'Calibrated Hit dQ/dx [ADC/cm]', 
                                       title="After re-calibration\n" + title,
                                       log=False,
                                       subtracted=True,
                                       area_norm=True,
                                       onebin=False)
            plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_depois.png', dpi=300)
            plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_depois.pdf')
            plt.close()
            
            chi2_depois[plane_num][-1].append(out2[1]/out2[2])
            
            out2 = my_plotter.plot_pot_normalised_var(variable='dqdx'+plane+"_cali",
                                       binning=binning, 
                                       categories=pdg_categories, 
                                       additional_selection=sel_function, 
                                       prediction_datasets=['bnb_dirt', 'bnb_nu'],
                                       xlabel=r'Calibrated Hit dQ/dx [ADC/cm]', 
                                       title="After re-calibration\n" + title,
                                       log=False,
                                       subtracted=True,
                                       area_norm=True,
                                       onebin=False,
                                       legend=False)
            plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_noleg_depois.png', dpi=300)
            plt.savefig(this_save_folder + 'plane{}/'.format(plane_num) + save_label + '_noleg_depois.pdf')
            plt.close()

/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/plotter.py:184: RuntimeWarning: invalid value encountered in true_divide
  bin_relative_errors_data = beam_on_y_err / beam_on_values
/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/plotter.py:184: RuntimeWarning: invalid value encountered in true_divide
  bin_relative_errors_data = beam_on_y_err / beam_on_values
/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/plotter.py:184: RuntimeWarning: invalid value encountered in true_divide
  bin_relative_errors_data = beam_on_y_err / beam_on_values
/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/plotter.py:184: RuntimeWarning: invalid value encountered in true_divide
  bin_relative_errors_data = beam_on_y_err / beam_on_values
/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/plotter.py:184: RuntimeWarning: invalid value encountered in true_divide
  bin_relative_errors_data = beam_on_y_err / beam_on_values
/home/nic/Dropbox/MicroBooNE/bnb_nue_analysis/lib/plotter.py:184: RuntimeWarning: inv

In [23]:
# summary plots of the chi2
plt.rcParams.update({'font.size': 10})

this_title_base = '\n' + title_base + '\nplane {}\nMicroBooNE Preliminary'

def chi2_plot(table, title, save_name):
    plt.pcolormesh(table)
    plt.title(title, loc='left')
    plt.colorbar()
    plt.xticks(range(len(parameters_bin_edges[1])), ['{:.2g}'.format(p) for p in parameters_bin_edges[1]])
    plt.yticks(range(len(parameters_bin_edges[0])), ['{:.2g}'.format(p) for p in parameters_bin_edges[0]])
    plt.xlabel("pitch [cm]")
    plt.ylabel("residual range [cm]")
    
    for i in range(len(parameters_bin_edges[1])-1):
        for j in range(len(parameters_bin_edges[0])-1):
            text = plt.text(i+0.5, j+0.5, "{:.3g}".format(table[j, i]),
                           ha="center", va="center", color="k")
    plt.tight_layout()
    plt.savefig(save_name + '.png', dpi=300)
    plt.savefig(save_name + '.pdf')
    plt.close()

for plane_num in range(3):
    chi2_av = np.array(chi2_avant[plane_num])
    chi2_dep = np.array(chi2_depois[plane_num])
    
    chi2_plot(chi2_av, title=r"$X^2$" + " Before re-calibration" + this_title_base.format(plane_num), 
                       save_name=this_save_folder + 'plane_{}_chi2_avant'.format(plane_num))
    
    chi2_plot(chi2_dep, title=r"$X^2$" + " After re-calibration" + this_title_base.format(plane_num), 
                       save_name=this_save_folder + 'plane_{}_chi2_depois'.format(plane_num))
    
    chi2_plot(chi2_dep-chi2_av, title=r"$X^2_{after} - X^2_{before}/ndof$" + this_title_base.format(plane_num), 
                       save_name=this_save_folder + 'plane_{}_delta_chi2'.format(plane_num))

    chi2_plot(chi2_dep/chi2_av, r"$X^2_{after}/X^2_{before}$" + this_title_base.format(plane_num), 
                       save_name=this_save_folder + 'plane_{}_chi2_ratio'.format(plane_num))